Primal problem:
$$\min c^T x \quad\text{subject to}\quad Ax=b, -x\leq 0.$$

Dual problem:
$$\max b^T y \quad\text{subject to}\quad A^Ty\leq c.$$

In [1]:
import numpy as np

In [2]:
# m: number of constraints
# n: number of variables
m = 50
n = 100

A = np.random.randn(m, n)
x0 = np.random.random(n)
b = A.dot(x0)
z = np.random.randn(m)
s = 10*np.random.random(n)

# This is to make sure that the dual problem is feasible in interior since A.T @ z < c.
c = A.T.dot(z) + s

In [3]:
def get_res(x, lambda_, v, tinv, grad_f = c):
    r_dual = Dh.dot(lambda_) + A.T.dot(v) + grad_f
    r_cent = -np.diag(lambda_)@h(x) - tinv
    r_pri = A.dot(x)-b
    return r_dual, r_cent, r_pri

def get_res_norm(x, lambda_, v, tinv):
    r_dual, r_cent, r_pri = get_res(x, lambda_, v, tinv)
    return np.linalg.norm(r_dual), np.linalg.norm(r_cent), np.linalg.norm(r_pri)

In [4]:
def primal_dual(f, grad_f, nabla_f, h, grad_h, x0, A, b, dom_f, MAXITERS=100, TOL=1e-8,alpha = 0.01, beta = 0.8, RESTOL=1e-8):
    MAXITERS = MAXITERS
    TOL = TOL
    RESTOL = RESTOL
    mu = 10
    alpha = alpha
    beta = beta
    gaps = []
    resdls = []
    x = x0.copy()   
    h = h
    Dh = grad_h
    lambda_ = -1 / (h(x))
    v = np.zeros(m)

    for iters in range(MAXITERS):
        # Surrogate duality gap
        ita = - h(x).dot(lambda_)
        gaps.append(ita)
        tinv = ita/(m*mu)

        r_dual, r_cent, r_pri = get_res(x, lambda_, v, tinv)
        resdls.append(np.linalg.norm(np.hstack([r_dual, r_pri])))
        # stopping criterion
        if ita < TOL and resdls[-1] < RESTOL:
            break
        
        sol = -np.linalg.solve(
            np.block([[np.zeros((n, n)), Dh.T, A.T],
                    [-np.diag(lambda_)@Dh, -np.diag(h(x)), np.zeros((n, m))],
                    [A, np.zeros((m, m)), np.zeros((m, n))]]),
            np.hstack([r_dual, r_cent, r_pri])
        )
        dx = sol[:n]
        dlambda_ = sol[n:n+n]
        dv = sol[-m:]

        # backtracking line search
        # The maximum step such that the new lambda_ is positive, i.e., feasible.
        step = min(1, 0.99/np.max(-dlambda_/lambda_))
        while True:
            x_new = x + step*dx
            if np.all(h(x_new) < 0):
                break
            step *= beta

        new_x = x + step*dx
        new_lambda_ = lambda_ + step*dlambda_
        new_v = v + step*dv
        
        # Old residual norm
        old_norm = sum(get_res_norm(x, lambda_, v, tinv))

        while sum(get_res_norm(new_x, new_lambda_, new_v, tinv)) > (1- alpha * step) * old_norm:
            step *= beta
            new_x = x + step*dx
            new_lambda_ = lambda_ + step*dlambda_
            new_v = v + step*dv

        x = new_x
        lambda_ = new_lambda_
        v = new_v



4.4195361683856536e-09 3.322773662846654e-14 30


## Solve by Log-barrier method
In this case, we have the problem
$$\min t c^T x-log(x)\quad \text{subject to}\quad  Ax=b$$

In [6]:
from functions import newton_eq

In [7]:
t = 10
x_inner = x0.copy()
while m/t > TOL:
    f = lambda x: t*c.dot(x) - np.sum(np.log(x))
    grad_f = lambda x: t*c - 1/x
    nabla_f = lambda x: np.diag(1/x**2)

    x_list, fx_list = newton_eq(f, grad_f, nabla_f, x_inner, A, b, dom_f=lambda x:np.all(x>0), MAXITERS=50, TOL=1e-5, alpha=0.001, beta=0.8)
    x_inner = x_list[-1]
    t *= mu